# Improving public policy by predicting car accident severity using machine learning
---

**Disclaimer:** _This notebook is part of the Coursera Capstone Project to complete the [IBM Certification in Data Science](https://www.coursera.org/professional-certificates/ibm-data-science). All project ideas are fictitious and serve only the purpose of developing a data science project._


Original dataset: [link_dataset](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv)  
Metadata: [link_metadata](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf)



## Table of Contents
---
### 1 - [Introduction/Business Problem](#business-und)

### 2 - [Data understanding and Data Preparation](#data-und-data-prep)

### 3 - [Modeling](#modeling)

### 4 - [Evaluation](#evaluation)

### 5 - [Conclusion](#conclusion)

<a id="business-und"><a/>

# 1 - Introduction/Business Problem
---- 


Predicting accident severity in US cities based on specific metrics can be a powerful tool to drive public policy and reduce overall accident rate. In Seattle – Washington, the Department of Transport/Traffic Management Division,  has been collecting data since 2004 about collisions in the metropolitan area with the objective of creating a complete database that represents the overall road accidents involving collisions in this city. The City Council of Seattle has the responsibility of approving the city's budget, and develops laws and policies intended to promote safety of Seattle's residents  . During every fiscal year, the city council discusses policies to improve road safety in Seattle by reducing the number of human injuries involved in those accidents. 
    
In this data science project, I proposed to develop a model that can distinguish accidents resulting in human injuries from accidents resulting in property damage-only. This prediction will be based on widely available metrics provided by the Department of transport and it can help to identify which factors may increase the risk for injury-related accidents and help develop actions to reduce those. The successful outcome of this project would be a model that can predict with accuracy accidents associated with human costs (i.e. high true positive rate), so that actions can be developed to minimize those costs. Examples of actions could include target advertising for road safety, improve road design, increase police deployment to secure roads, increase fines for reckless driving, among others. 


In [ ]:
##### Import necessary packages
import sklearn
import numpy as np
import scipy
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-dark')
plt.rcParams['axes.titlesize'] = 14

In [ ]:
# specify the location of the data
data_url = 'https://github.com/joseferncruz/coursera-capstone/raw/master/datasets/collisions_data.csv'

load the data into a dataframe
df_raw = pd.read_csv(
    data_url,
    parse_dates=['INCDTTM', 'INCDATE'], # Parse dates to datetime objects 
    usecols=[
        'SEVERITYCODE', 'X', 'Y', 'ADDRTYPE', 'PERSONCOUNT', 'PEDCOUNT',
        'PEDCYLCOUNT', 'VEHCOUNT', 'INCDTTM', 'JUNCTIONTYPE', 'WEATHER',
        'ROADCOND', 'LIGHTCOND', 'CROSSWALKKEY', 'INCDATE',
    ],
    low_memory=False,
);

df_raw.drop(columns='INCDATE', inplace=True)

<a id="data-und-data-prep"><a/>

# 2 - Data understanding and Data Preparation
---
    
To develop my data project, I will use data about collisions provided by Seattle Police Department and recorded by Traffic Records . This dataset includes all types of collisions happening at intersection or mid-block of a road segment since 2004 and contains information about many important factors such as road condition and lightning conditions, weather, segment of the road involved (among other) and associated with each accident there is a variable that represents the outcome severity with 2 values: type 1 – property damage-only and type 2 – Injury-related. By using this information, I will develop a classification model aiming at predicting the severity outcome of the accident, with particular emphasis at predicting type 2 accidents.  I will focus on optimizing the model to get a high true positive rate of detection of type 2 accidents and the results obtained from this model could guide actions to decrease the occurrence of these accidents. 

In order to solve this classification problem, I will use the following features from the original dataset:


|Attributes| Description|  
|---:|:----|  
|SEVERITYCODE|A code that corresponds to the severity of the collision|  
| X | GPS Longitude coordinate |
| Y | GPS Latitude coordinate | 
|ADDRTYPE|Collision address type|
|PERSONCOUNT|The total number of people involved in the collision|
|PEDCOUNT|The number of pedestrians involved in the collision |
|PEDCYLCOUNT| The number of bicycles involved in the collision. |
|VEHCOUNT|The number of vehicles involved in the collision|
|INCDTTM|The date and time of the incident.|
| JUNCTIONTYPE| Category of junction at which collision took place |
|WEATHER|A description of the weather conditions during the time of the collision|
|ROADCOND|The condition of the road during the collision|
|LIGHTCOND|The light conditions during the collision|
|CROSSWALKKEY| A key for the crosswalk at which the collision occurred |


Let's start by exploring the features and characteristics of this dataset.

In [ ]:
print(f'This dataset has {df_raw.shape[0]} rows and {df_raw.shape[1]} columns, including one target column.')

In [ ]:
# Drop columns not used in this case study
df = df_raw.copy()

# display some information about the datatypes and number of entries associated with the full raw dataset
df_raw.info()

By exploring the metadata, it is possible to notice that some columns have large quantities of missing data or lack information (Not Enough Information or NEI). Also our model should be able to predict accident severity based on features (ie information) that can be measured in real time or within an hour range (eg weather, road condition, etc).

The first step is to deal with the missing/unknown information from columns in order to reduce the uncertainty around certain features.

In [ ]:
# Remove entries with missing information

# Remove missing/uncertain values from WEATHER
df['WEATHER'].replace(['Unknown', 'Other'], 'uncertain', inplace=True)
df['WEATHER'].fillna(value='uncertain', inplace=True)

# Remove missing/uncertain values from ROADCOND
df['ROADCOND'].replace(['Unknown', 'Other'], 'uncertain', inplace=True)
df['ROADCOND'].fillna(value='uncertain', inplace=True)

# Remove missing/uncertain values from LIGHTCOND
df['LIGHTCOND'].replace(['Unknown', 'Other', 'Dark - Unknown Lighting'], 'uncertain', inplace=True)
df['LIGHTCOND'].fillna(value='uncertain', inplace=True)

# JUNTIONTYPE
df['JUNCTIONTYPE'].replace(['Unknown'], 'uncertain', inplace=True)
df['JUNCTIONTYPE'].fillna(value='uncertain', inplace=True)

# ADDRTYPE
df[df.ADDRTYPE.isna()] = 'NaN'
df['ADDRTYPE'].replace(['NaN'], 'uncertain', inplace=True)

# Drop NaN from target variable.
df['SEVERITYCODE'].replace('NaN', np.nan, inplace=True)
df = df.loc[~df.SEVERITYCODE.isna(), :]

# GPS coords - substitute missing GPS coords with median value.
df[['X', 'Y']].replace('NaN', np.nan, inplace=True)
df['X'].fillna(np.median(df.X), inplace=True)
df['Y'].fillna(np.median(df.Y), inplace=True)

# Consider only complete years
#Extract the hour
df['YEAR'] = df['INCDTTM'].dt.year
# remove faulty entries (ie not correct hour record)
cond = df['YEAR'].isin([2020])
df = df.loc[~cond, :].copy()


In [ ]:
# Display information about the dataset without missing data or incomplete information.
df.info()

The `SEVERITYCODE` target variable can take 2 values (1/2):
- 1: **Property Damage**  
- 2: **Injury**




In [ ]:
# How many observations per target category exists?
for category, counts in df.SEVERITYCODE.value_counts().items():
    print(f"There are {counts} elements in target value {category}.")
    
# Print normalized count in each category of severity.
norm_count = df.SEVERITYCODE.value_counts(normalize=True).to_dict()
print(f"The dataset is devided into{norm_count.get(1)*100: 0.2f}% Severity 1 and{norm_count.get(2)*100: 0.2f}% Severity 2 observations.")

As it is possible to observe, our dataset is quite imbalanced and this poses a challenge to classification algorithms. For now, we will consider the full dataset (i.e. records without missing or ambiguous information) and I will address later the problem of the imbalanced data.

## Location of type 1 and type 2 severity events in the city of Seattle
---

Since we have GPS Data, the first step will be to cluster the city of Seattle into 10 different spatial clusters and identify those clusters. This information can also be relevant for the sponsors to target their actions to reduce accident severity.

In [ ]:
df.columns

### Divide the city of Seattle into 12 clusters

In [ ]:
from sklearn.cluster import KMeans

# Initiate KMeans for 10 clusters (ie putative city areas)
kmeans = KMeans(n_clusters = 12, init ='k-means++', random_state=32)

# Get coords dataframe
coords = df[['X', 'Y']]

# Fit the model
kmeans.fit(coords) # Compute k-means clustering.

# get the labels
labels = kmeans.predict(coords) # Labels of each point

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 10), sharey=True)

# Set condition to access both types of severity independently
cond = df['SEVERITYCODE'] == 1

# Class 1
ax[0].plot(df['X'][cond], df['Y'][cond], '.', ms=2, alpha=0.5, color='black')
ax[0].set(ylabel='Latitude')
ax[0].legend(labels=['Severity 1'])

# Class 2
ax[1].plot(df['X'][~cond], df['Y'][~cond], '.', ms=2, color='black', alpha=0.5)
ax[1].legend(labels=['Severity 2'])

# Both 1 and 2
for event_type in [1, 2]:
    ax[2].plot(df['X'][df['SEVERITYCODE'] == event_type], df['Y'][df['SEVERITYCODE'] == event_type], '.', ms=3, alpha=0.5)
ax[2].legend(labels=['Severity 1', 'Severity 2'], loc='upper right')

# Complete plot info
for idx in range(3):
    ax[idx].set(title='Seattle')
    ax[idx].set(xlabel='Longitude')
    ax[idx].tick_params('x', labelrotation=45)
    
    coords.plot.scatter(x = 'X', y = 'Y', c=labels, s=50, cmap='viridis', ax=ax[idx], alpha=0.5)

In [ ]:
# Create a new feature: `SECTOR_ID` with labels from the clustering algorithm.
df['SECTOR_ID'] = labels

In [ ]:
# Check the total numbe of severity 1 and 2 events for each sector
result = df.groupby(['SEVERITYCODE'])['SECTOR_ID'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()
result.sort_values('NORM_COUNT', inplace=True)

In [ ]:
sns.barplot(x='SECTOR_ID', y='NORM_COUNT', data=result, hue='SEVERITYCODE', )

It seems that Sector 4 and 8 have a higher proportion of accident severity 2. This information can be used with one-hot encoder to create 2 more variables `SECTOR_4` and `SECTOR_8`

In [ ]:
df['SECTOR_4'] = df['SECTOR_ID'].apply(lambda x: 1 if x == 4 else 0)
df['SECTOR_8'] = df['SECTOR_ID'].apply(lambda x: 1 if x == 8 else 0)

## When do events of specific severities happen more often?
---
In order to answer this question, we need to create new time features from the time feature in `INCDTTM`.

In [ ]:
# Extract the week day
df['WEEKDAY'] = df['INCDTTM'].dt.weekday

# Extract the hour
df['HOUR'] = df['INCDTTM'].dt.hour

# Extrack the month
df['MONTH'] = df['INCDTTM'].dt.month

# Extract the year
df['YEAR'] = df['INCDTTM'].dt.year

# Display top 3 columns
df.head(3)

### Evolution of the total number of accidents for each category since 2004:

In [ ]:
# Extract monthly normalized count of events for each category
result = df.groupby('SEVERITYCODE')['YEAR'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()
sns.barplot(x='YEAR', y='NORM_COUNT', data=result, hue='SEVERITYCODE', ax=ax)

ax.set(ylim=(0, 0.15), title='Yearly Accident variation for each Severity Category');
ax.tick_params('x', labelrotation=45)

It seems that accidents of type 2 tend to be more frequent since 2016. It is also interesting to notice an overall decrease in the number of accidents over the years. We can create a variable `SINCE_2016` that captures this information:

In [ ]:
df['SINCE_2016'] = df['YEAR'].apply(lambda x: 1 if x >= 2016 else 0)

In [ ]:
# Extract monthly normalized count of events for each category
result = df.groupby('SEVERITYCODE')['MONTH'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()
sns.barplot(x='MONTH', y='NORM_COUNT', data=result, hue='SEVERITYCODE', ax=ax)

month_labels = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
ax.set(ylim=(0, 0.15), xticklabels=month_labels, title='Monthly Accident proportion in each Severity Category');

It seems that **JULY** and **AUGUST** have a (low, but still) slightly higher proportion of accidents in the category 2. We could use this information to create one additional one-hot encoded variable `SUMMER`.

In [ ]:
df['SUMMER'] = df['MONTH'].apply(lambda x: 1 if x in [6, 7] else 0)

In [ ]:
# Show the proportion.
df.groupby('SEVERITYCODE')['SUMMER'].value_counts(normalize=True)

## What about daily the variation in accident severity? Is there a particular time of the day with a higher proportion of accidents belonging to a specific severity type?
----


In [ ]:
# Estract normalized count of each severity category for each hour
result = df.groupby('SEVERITYCODE')['HOUR'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()

fig, ax = plt.subplots()

sns.barplot(x='HOUR', y='NORM_COUNT', data=result, hue='SEVERITYCODE', ax=ax)

ax.set(title='Hourly accident proportion in each severity Category')

It seems that during the rush hour (7-8AM, 4-6PM) there is a higher proportion of accidents belonging to category 2, particularly during the rush hour (5-7PM). Let's capture this variation in another one-hot encoded variable `RUSH_HOUR`.

In [ ]:
df['RUSH_HOUR'] = df['HOUR'].apply(lambda x: 1 if x in [7, 8, 16, 17, 18] else 0)

df.groupby('SEVERITYCODE')['RUSH_HOUR'].value_counts(normalize=True)

## What is the average human cost associated with each city sector?
---
In order to maximize the clustering information, it would be good to find a measure of human cost that could be used to describe each individual group (ie `SECTOR_ID`). One way to do this is to explore a measurement of the ratio between humans vs vehicles involved.


In [ ]:
# Create a single column that agregates all the non-motorized human involvement
df['TOTAL_HUMAN'] = df[['PERSONCOUNT', 'PEDCYLCOUNT', 'PEDCOUNT']].sum(axis=1)
df['TOTAL_HUMAN'].head()

## What is the relationship between total number of people involved and total number of vehicles involved?  
---

Before dwelling into the relation, let's have a look at the distribution of values within each variable.

In [ ]:
# Some stats about severity and location
df.groupby(['SEVERITYCODE'])[['VEHCOUNT', 'TOTAL_HUMAN']].agg([np.mean, np.min, np.max])

It seems that a higher number of people is associated with a severity of type 2, although the mean is only slightly lower. Let's look at the distribution.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Normalized log distribution of People involved in Accidents.')

bins = np.linspace(0, 100, 20)

ax[0].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==1], bins=bins, color='#1f77b4', label='Severity 1', log=True, density=True);
ax[1].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==2], bins=bins, color='#ff7f0e', label='Severity 2', log=True, density=True);

ax[2].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==2], log=True, bins=bins, color='#ff7f0e', label='Severity 2', density=True);
ax[2].hist(df['TOTAL_HUMAN'][df['SEVERITYCODE']==1], log=True, bins=bins, color='#1f77b4', label='Severity 1', density=True);


for idx in range(3):
    ax[idx].legend()
    ax[idx].set(ylabel='normalized log count', xlabel='People')

According to the plot above (normalize to total count and in logarithmic scale) it is possible to observe that category 2 accidents involve frequently more humans than type 1.


In [ ]:
df['VEHCOUNT'] = df.VEHCOUNT.astype(int)

In [ ]:
result = df.groupby(['SEVERITYCODE', 'SECTOR_ID']).mean()[['TOTAL_HUMAN', 'VEHCOUNT']].reset_index()


In [ ]:
# Plot the relation between total number of people involved vs vehicles.
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Relation between Vehicle Number and Humans involved.')

sns.regplot(
    x='TOTAL_HUMAN',
    y='VEHCOUNT',
    data=result,
    order=1,
    ax=ax[0]
)
ax[0].set(
    ylabel='Mean Vehicle Number',
    xlabel='Mean Human Number',
    title='Linear Regression between vehicles and humans',
    xlim=(2, 3.5)
)

sns.residplot(
    x='TOTAL_HUMAN',
    y='VEHCOUNT',
    data=result,
    order=1,
    ax=ax[1]
)
ax[1].set(
    title='Residual plot',
    ylabel='Residuals from the model',
    xlabel='',
    ylim=(-0.15, 0.15),
);

### Are the two variables correlated?
----

In [ ]:
slope, intercept, rvalue, pvalue, stderr = scipy.stats.linregress(result[['TOTAL_HUMAN', 'VEHCOUNT']])

print(f"The correlation coeff (person) is: {rvalue: 0.2f} and the p-value associated is {pvalue: 0.3f}")

The two variables are negatively correlated, meaning that there is a higher number of people associated with accidents involving fewer vehicles. 

## What is the ratio between total number of people involved and number of vehicles?
---

Let's now calculate the ratio between people involved and vehicles involved for each observation in the dataset.

In [ ]:
# Add small value to VEH count to avoid ZeroDivisionError 
df[['VEHCOUNT']] = df[['VEHCOUNT']] + 0.001

# Divide total_human by veh_count
df['RATIO_HUMAN/VEH'] = df['TOTAL_HUMAN'].div(df['VEHCOUNT'])

Now let's check the mean of the `RATIO_HUMAN/VEH` of each Cluster and for each severity category.

In [ ]:
human_cost = df.groupby(['SEVERITYCODE', 'SECTOR_ID']).mean()['RATIO_HUMAN/VEH'].to_frame().reset_index()

human_cost.head()

Let's Create a new feature called `SECTOR_HUMAN_COST` which stores the score of human cost for each sector.

In [ ]:
severity_code_1 = human_cost[human_cost['SEVERITYCODE']==1][['SECTOR_ID', 'RATIO_HUMAN/VEH']]
severity_code_1 = severity_code_1.to_dict().get('RATIO_HUMAN/VEH')

severity_code_2 = human_cost[human_cost['SEVERITYCODE']==2][['SECTOR_ID', 'RATIO_HUMAN/VEH']].reset_index()
severity_code_2 = severity_code_2.to_dict().get('RATIO_HUMAN/VEH')

In [ ]:
# 
df['SECTOR_HUMAN_COST_a'] = df[df['SEVERITYCODE']==1]['SECTOR_ID'].replace(severity_code_1)
df['SECTOR_HUMAN_COST_a'].fillna(0, inplace=True)

In [ ]:
df['SECTOR_HUMAN_COST_b'] = df[df['SEVERITYCODE']==2]['SECTOR_ID'].replace(severity_code_2)
df['SECTOR_HUMAN_COST_b'].fillna(0, inplace=True)

In [ ]:
df['SECTOR_HUMAN_COST'] = df['SECTOR_HUMAN_COST_a'] + df['SECTOR_HUMAN_COST_b']

In [ ]:
fig, ax = plt.subplots( figsize=(10, 5))
sns.barplot(x='SEVERITYCODE', y='SECTOR_HUMAN_COST', data=df, hue='SECTOR_ID', ax=ax)

Create an uniform value to define each cluster.


In [ ]:
net_human_cost = human_cost['RATIO_HUMAN/VEH'][human_cost['SEVERITYCODE']==2].reset_index(drop=True)\
                 - human_cost['RATIO_HUMAN/VEH'][human_cost['SEVERITYCODE']==1]

In [ ]:
# Set up new variable 
df['SECTOR_HUMAN_COST'] = df['SECTOR_ID'].replace(net_human_cost.to_dict())

In [ ]:
# Plot the results 
sns.barplot(
    x='SECTOR_ID',
    y='SECTOR_HUMAN_COST',
    data=df
)

In [ ]:
df['TOTAL_INTERV'] = df['TOTAL_HUMAN'] + df['VEHCOUNT']

## Are specific types of road structures involved in specific types of events?
---
Among the data chosen to build the model, we can find information about types of collision addresses: Intersection and Block.


In [ ]:
result = df.groupby(['SEVERITYCODE'])['ADDRTYPE'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()

fig, ax = plt.subplots()
sns.barplot(
    data=result, 
    x='ADDRTYPE', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
    ax=ax
)

ax.set(
    title='Accident severity and collision address type', 
    ylabel='Normalized count', 
    xlabel='Collision address type',
    ylim=(0, 1)
)

It seems that Severity type 2 accidents happen in equal proportions in both Blocks and Intersections whereas type 1 accidents tend to happen more in Blocks.  

## What about the road condition? How does road condition relate to accident severity?
---
There are several different attributes in terms of road condition, for instance:


In [ ]:
print('Different states of road conditions', df.ROADCOND.unique())

In [ ]:
result = df.groupby(['SEVERITYCODE'])['ROADCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='ROADCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)
ax.tick_params('x', labelrotation=45)
ax.set(title='Road conditions and accident severity', ylabel='normalized count', xlabel='')

It seems that there are more accidents of category 2 during dry and wet conditions. This apparent paradox could be explained by overconfidence during dry and wet conditions that could induce reckless driving behaviors.

In [ ]:
dry = ['Dry']
wet = ['Wet', 'Ice', "Snow/Slush", 'Sand/Mud/Dirt', 'Standing Water', 'Oil']


for word in wet:
    df['ROADCOND'] = df['ROADCOND'].replace(word, 'wet')
for word in dry:
    df['ROADCOND'] = df['ROADCOND'].replace(word, 'dry')
    
df.groupby('SEVERITYCODE').ROADCOND.value_counts(normalize=True)

In [ ]:
df.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True)

In [ ]:
result = df.groupby(['SEVERITYCODE'])['ROADCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='ROADCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)
ax.tick_params('x', labelrotation=45)
ax.set(title='Road conditions and accident severity', ylabel='normalized count', xlabel='')

Another important component that can play a role in accident severity is the lighting conditions at the time of the accident.

## Is there any relationship between accident severity and luminosity at the time of the accident?
---
Similarly to road condition, there are several different values associated with this feature.


In [ ]:
df.groupby('SEVERITYCODE').LIGHTCOND.value_counts(normalize=True)

In [ ]:
result = df.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='LIGHTCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)
ax.tick_params('x', labelrotation=45)
ax.set(title='Road conditions and accident severity', ylabel='normalized count', xlabel='')

Let's bin the values by good and bad visibility conditions.


In [ ]:
# Group by similar condtions
day_light = ['Daylight']
low_light = ['Dark - Street Lights On', 'Dusk', 'Dawn', 'Dark - No Street Lights', 'Dark - Street Lights Off']

for word in day_light:
    df['LIGHTCOND'] = df['LIGHTCOND'].replace(word, 'light-good')
for word in low_light:
    df['LIGHTCOND'] = df['LIGHTCOND'].replace(word, 'light-bad')
    
df.LIGHTCOND.unique()

In [ ]:
result = df.groupby(['SEVERITYCODE'])['LIGHTCOND'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='LIGHTCOND', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 1), title='Light conditions and accident severity', ylabel='normalized count', xlabel='')

It seems that good lightning conditions have a higher proportion of accidents of type 2 (i.e. injury-related). It could be explained by the higher number of people on the streets during daylight.



While we have observed that Intersections have a higher proportion of type 1 accidents, We can explore the feature `JUNCTIONTYPE` to extract more insights about the properties of the accidents.


In [ ]:
# Junction Type
df.groupby('SEVERITYCODE').JUNCTIONTYPE.value_counts(normalize=True)

It seems that type 1 Severity Accidents happen 50% of the times at Mid-Blocks whereas type 2 accidents happen with ~48% of the times at Intersection (intersection related). Since getting higher positive rate of type 2 accidents is our priority, I will use this information to create a one-hot encoded feature `INTERSECTION_RELATED` with this information:



In [ ]:
df['INTERSECTION_RELATED'] = df['JUNCTIONTYPE'].apply(lambda x: 1 if x == 'At Intersection (intersection related)' else 0)


## How is the weather forecast related to accident severity?
---
The weather is also an important aspect of safe driving conditions and can have a strong impact on the type of accidents that can occur. In our dataset, weather can have the following values:


In [ ]:
df['WEATHER'].unique()

In [ ]:
result = df.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='WEATHER', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 0.7), title='Weather and accident severity', ylabel='normalized count', xlabel='');
ax.tick_params('x', labelrotation=30)

In [ ]:
df.groupby('SEVERITYCODE').WEATHER.value_counts(normalize=True)

The vast majority of the accidents (>65%) happen during periods of Clear sky. Let's bin the different categories in: `sunny`, `cloudy` and `rainy-snow`.

In [ ]:
# Eeather categories
sunny = ['Clear', ]
cloudy = ['Overcast', 'Fog/Smog/Smoke', 'Partly Cloudy', 'Blowing Sand/Dirt', 'Severe Crosswind', ]
rainy_snow = ['Raining', 'Snowing', 'Sleet/Hail/Freezing Rain', ]

for word in sunny:
    df['WEATHER'] = df['WEATHER'].replace(word, 'sunny')
for word in cloudy:
    df['WEATHER'] = df['WEATHER'].replace(word, 'cloudy')
for word in rainy_snow:
    df['WEATHER'] = df['WEATHER'].replace(word, 'wet')

In [ ]:
result = df.groupby(['SEVERITYCODE'])['WEATHER'].value_counts(normalize=True).to_frame('NORM_COUNT').reset_index()


fig, ax = plt.subplots()

sns.barplot(
    data=result, 
    x='WEATHER', 
    y='NORM_COUNT', 
    hue='SEVERITYCODE',
)

ax.set(ylim=(0, 0.7), title='Weather and accident severity', ylabel='normalized count', xlabel='weather forecast');

In agreement with the road condition variable, during sunny days (ie dry road) the amount of accidents of type 2 is higher compared to type one. It seems that wet road conditions are also associated with higher type 2 accidents.

## Accidents at Crosswalks
---
Finally and to complete this basic EDA, let's have a look at the `CROSSWALKKEY` feature. This feature holds the key of the crossroad that is associated with an accident. Let's investigate whether accidents at crosswalks are associated with specific types of accidents.


In [ ]:
# Transform the variable in 1 where there is an event at crossroad (ie accident) or 0 elsewhere.
df['CROSSWALKKEY'] = df['CROSSWALKKEY'].apply(lambda x: 0 if x == 0 else 1)

df.groupby('SEVERITYCODE')['CROSSWALKKEY'].value_counts(normalize=True)

It seems that the majority of the accidents __do not__ take place at crossroads. However, severity 2 accidents tend to be associated in higher proportion to crossroads compared to severity 1 accidents.

Select and prepare data for modeling.
---

In [ ]:
df = df[['SEVERITYCODE', 'ADDRTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'CROSSWALKKEY',
         'SECTOR_4', 'SECTOR_8', 'WEEKDAY', 'MONTH', 'INTERSECTION_RELATED',
         'RUSH_HOUR', 'SUMMER', 'TOTAL_HUMAN', 'SECTOR_HUMAN_COST']]

df.info()

### One-hot encoding of categorical data
---

In [ ]:
df = pd.get_dummies(df, drop_first=True)
df.info()

### Balance the dataset
---
#### Implementing a resampling method of the majority class.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler # https://imbalanced-learn.readthedocs.io/en/stable/api.html

# Create resampling object 
rus = RandomUnderSampler(replacement=False, random_state=32, sampling_strategy='majority')

# create target bool array to subset dataset
target_cond = df.columns == 'SEVERITYCODE'

# Undersample the data
FEATURES, TARGET = rus.fit_sample(df.loc[:, ~target_cond], df.loc[:, target_cond])

# Reassemble the dataset for preprocessing
FEATURES['SEVERITYCODE'] = TARGET

# Reassign variable to df
df = FEATURES.copy()

# Show the information about undersampled dataset
df.info()

<a id="modeling"><a/>

# Modeling
---

In this problem, I intend to build a binary classifier model based on previous observations. Thus, this problems represents a supervised learning problem and in order to tackle my classification problem I will use 3 types of machine learning algorithms:

- Logistic Regression
- Gradient Boosting Classifier
- Random Forest ensemble

    

In [ ]:
# Import models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Import preprocessing tools and model selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint, truncnorm, norm

## Split data into Train and Test datasets

In [ ]:
from sklearn.model_selection import train_test_split

# Features used to generate the model
FEATURES = df.loc[:, df.columns != 'SEVERITYCODE']

# What is intended to be predicted
TARGET = df.loc[:, ~(df.columns != 'SEVERITYCODE')].replace({1: 0, 2: 1}).to_numpy().flatten()

# Split the data into train/test data
X_train, X_test, y_train, y_test = train_test_split(FEATURES, TARGET, random_state=32, test_size=0.40)

## Scale the data

This is an important step for many algorithms. In order to implement cross validation, I will build a pipeline using a `sklearn.pipeline` object to ensure that there is no information leakage during cross validation.

## Create dummy classifier
The purpose of the dummy classifier is to assess how much of my model prediction is attributed to chance. For that I will implement and use a dummy classifier that chooses the most frequent class.

In [ ]:
from sklearn.dummy import DummyClassifier

# Create pipeline
steps = [
    ('scaler', StandardScaler()), 
    ('dummy_clf', DummyClassifier(strategy='most_frequent', random_state=32)),
]

dummy_clf = Pipeline(steps)

# Fit Data
dummy_clf.fit(X_train, y_train)

# Predict
y_dummy = dummy_clf.predict(X_test)

## Generalized Linear Model for Classification: Logistic Regression.

Let's use grid search cross validation chained with a scaler object to find the best params to our logistic regression model.

In [ ]:
# list of steps for the pipeline
steps = [
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(random_state=32, max_iter=300))
]

# Assemble the pipeline for the logistic regression
logreg = Pipeline(steps)

# Implement stratified Cross Validation
Kfold = KFold(n_splits=5)

# Find the best parameters using gridsearch cross validation
params_grid = [{ 
    'logreg__solver': ['liblinear'],
    'logreg__penalty': ['l2'],
    'logreg__C': [0.01, 0.1, 1, 10]
},
    {
        'logreg__solver': ['saga'],
        'logreg__penalty': ['l1', 'l2'],
        'logreg__C': [0.01, 0.1, 1, 10]
    }
]

# Instantiate grid search cross validation object
gs_logreg = GridSearchCV(logreg, params_grid, cv=Kfold);

# Fit the model
gs_logreg.fit(X_train, y_train);

In [ ]:
print(f'The best score for the GridSearch CrossValidation was{gs_logreg.best_score_: 0.2f}')

In [ ]:
# print the best params
gs_logreg_best_params = gs_logreg.best_params_

print(gs_logreg_best_params)

In [ ]:
# The GridSearch Cross Validation automatically refits the best params so we simply need to make predictions 
y_logreg = gs_logreg.predict(X_test)

## Random Forest Classifier

Strategy:
- Pipeline with scaler and estimator
- RandomizedSearch coupled with stratified CrossValidation
-  Use estimtor refit with best parameters for prediction

In [ ]:
# Initiate the Logistic regression model
steps = [
    ('scaler', StandardScaler()),
    ('forest', RandomForestClassifier(random_state=32, n_jobs=-1))
]

# Build pipeline for forest
forest = Pipeline(steps)

# Implement stratified Cross Validation
kfold = KFold(n_splits=5)

# Find the best parameters using gridsearch cross validation
param_distributions = { 
    'forest__n_estimators': randint(100, 500),
    'forest__max_depth': randint(1, 6),
    'forest__max_features': truncnorm(a=0, b=1, loc=0.25, scale=0.1),
}

# Instantiate grid search cross validation object
gs_forest = RandomizedSearchCV(forest, param_distributions, cv=Kfold);

In [ ]:
# Fit the model
gs_forest.fit(X_train, y_train);

In [ ]:
gs_forest.best_params_

In [ ]:
# Predict
y_forest = gs_forest.predict(X_test)

### Gradient Boosting Classifier

Strategy:
- Pipeline with scaler and estimator
- RandomizedSearch coupled with stratified CrossValidation
-  Use estimtor refit with best parameters for prediction

In [ ]:
# Initiate the Logistic regression model
steps = [
    ('scaler', StandardScaler()),
    ('gbrt', GradientBoostingClassifier(random_state=32))
]

# Build pipeline for forest
gbrt = Pipeline(steps)

# Implement stratified Cross Validation
kfold = KFold(n_splits=5)

# Find the best parameters using gridsearch cross validation
param_distributions = { 
    'gbrt__n_estimators': randint(100, 300),
    'gbrt__max_depth': randint(1, 5),
    'gbrt__learning_rate': uniform(0, 1),
}

# Instantiate grid search cross validation object
gs_gbrt = RandomizedSearchCV(gbrt, param_distributions, cv=Kfold);

In [ ]:
# Fit the model
gs_gbrt.fit(X_train, y_train);

In [ ]:
# predict
y_gbrt = gs_gbrt.predict(X_test)

<a id="evaluation"><a/>

# Evaluation
    
The objective of this project is to develop a model that predicts accurately accidents in the category 2, so that new policies can be developed to minimize those types of accidents. Thus, we want to have a model with a high True Positive Rate for Category 2 accidents.

For this binary classification problem, I will use precision and recall measurements, associated F1 scores and I will have a look at the receiver operating characteristic (ROC) curves.

In [ ]:
# Import metrics
from sklearn.metrics import f1_score, classification_report, plot_confusion_matrix, plot_roc_curve, plot_precision_recall_curve

### F1 Scores

Let's look at the different F1 scores for each estimator:

In [ ]:
print(f'   Logistic Regression: F1 Score on test set: {f1_score(y_test, y_logreg,): 0.2f}')
print(f'Random Forest Ensemble: F1 Score on test set: {f1_score(y_test, y_forest,): 0.2f}')
print(f'     Gradient Boosting: F1 Score on test set: {f1_score(y_test, y_gbrt,): 0.2f}')
print(f'      Dummy Classifier: F1 Score on test set: {f1_score(y_test, y_dummy,): 0.2f}')

### Classification Report

To better undertand the F1 scores above let's have a look at the `classification_report`.

In [ ]:
target_names = ['Severity 1', 'Severity 2']

print(f"Logistic Regresion\n--------------------\n{classification_report(y_test, y_logreg, target_names=target_names)}")
print("Random Forest\n--------------------\n", classification_report(y_test, y_forest, target_names=target_names))
print("Gradient Boosting\n---------------------\n", classification_report(y_test, y_gbrt, target_names=target_names))
print("Dummy\n---------------------\n", classification_report(y_test, y_dummy, target_names=target_names))

### Confusion Matrix

By analysing the classification report, we can observe that the Random Forest Ensemble model provides the best recall (ie True Positive Rate) for severity 2 accidents which are the accidents that we intend to primarily target and reduce. Let's have a look at the confusion matrix to have a better visual understanding:

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(1, 4, figsize=(25, 5))

for idx, (name, estimator) in enumerate({'Logistic Regression': gs_logreg.best_estimator_,
                                         'Random Forest': gs_forest.best_estimator_,
                                         'Gradient Boosting': gs_gbrt.best_estimator_,
                                         'Dummy Classifier': dummy_clf,
                                        }.items()):
    plot_confusion_matrix(estimator, X_test, y_test, ax=ax[idx], normalize='true', display_labels=target_names, )
    
    ax[idx].set(title=f'{name}')

### Precision/recall curves

Another way to evaluate the best model is to have a look at the precision/recall curves. In a classification task, we can look at the tradeof of maximizing:
- recall (reduction of false negatives > accidents predicted to be of severity 1 but that are instead of severity 2)
or
- precision (reduction of false positive > accidents predicted to be of severity 2 but that are of severity 1)

In your example, we thrive to reduce false negatives (reduce the so called type 2 error) as the human cost associated with a false negative (failing to predict a type 2 severity accident) is higher than a false positive (prediction of a type 2 accident that is indeed false).

In [ ]:
# Precision Recall Curves
fig, ax = plt.subplots(1, 4, figsize=(25, 5),)
for idx, (name, estimator) in enumerate({'Logistic Regression': gs_logreg.best_estimator_,
                                         'Random Forest': gs_forest.best_estimator_,
                                         'Gradient Boosting': gs_gbrt.best_estimator_,
                                         'Dummy Classifier': dummy_clf,
                                        }.items()):
    plot_precision_recall_curve(estimator, X_test, y_test, ax=ax[idx])
    ax[idx].set(title=f'{name}')

### Receiver Operating Curves

Another way to evaluate the performance of our classifier is to look at the receiver operating curves. An ideal ROC curve should have a high area under the curve (AUC) and should be closer to the upper left corner (High True Positive Rate: Most accidents correctly classifier in their categories + Low False Positive Rate: Small number of accidents mispredicted).

As it is possible to observe, both Random Forest and Gradient Boosting algorithms have the higher AUC, however based on our previous analysis, the best estimator for our problem is the Random Forest. As expected, the dummy classifier has AUC of 0.50 meaning that predictions are completely random.


In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(25, 5))
for idx, (name, estimator) in enumerate({'Logistic Regression': gs_logreg.best_estimator_,
                                         'Random Forest': gs_forest.best_estimator_,
                                         'Gradient Boosting': gs_gbrt.best_estimator_,
                                         'Dummy Classifier': dummy_clf,
                                        }.items()):
    plot_roc_curve(estimator, X_test, y_test, ax=ax[idx])
    ax[idx].plot([0, 1], [0, 1], 'k--')
    ax[idx].set(title=f'{name}')

<a id="conclusion"><a/>

# Conclusion



In this mode, I manage to develop a model that is able to correctly classify severity 2 accidents (i.e. injury-related) with a F1 score of 0.69. Particularly, this model predicted correctly accidents of type 2 ~70% of the times and thus it is suitable to guide decision making processes intending at reducing this type of accident. As the model and respective pe designed to reduce the number of accidents and improve the road safety and overall city safety.
